# Ensembling Algorithms

In [1]:
!pip install scikeras

In [2]:
!pip install tensorflow==2.14.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 959.0 kB/s eta 0:00:000:0100:09m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.2 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 2.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 3.0.5
    Uninstalling keras-3.0.5:
      Successfully uninstalled keras-3.0.5
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully uninstalled google-auth-oauthlib-1.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboar

In [3]:
import os
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import applications, Model
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier
import sklearn
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer
from PIL import Image

warnings.simplefilter('ignore')

2024-03-22 22:51:29.956756: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 22:51:29.956815: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 22:51:29.956850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Import data

In [4]:
train_data_dir = '/kaggle/input/landmark-identification/FinalData/Training_Data'
validation_data_dir = '/kaggle/input/landmark-identification/FinalData/Validation_Data'
test_data_dir= '/kaggle/input/landmark-identification/FinalData/Test_Data'

train_labels_file = '/kaggle/input/landmark-identification/Labels/training_labels.npy'
validation_labels_file = '/kaggle/input/landmark-identification/Labels/validation_labels.npy'
test_labels_file = '/kaggle/input/landmark-identification/Labels/test_labels.npy'

img_width, img_height = 224, 224
num_classes = 10

In [5]:
train_labels = np.load(open(train_labels_file, 'rb'))
validation_labels = np.load(open(validation_labels_file, 'rb'))
test_labels = np.load(open(test_labels_file, 'rb'))

In [6]:
print('Training Data : ' + str(len(train_labels)) + ' Images')
print('Validation Data : ' + str(len(validation_labels)) + ' Images')
print('Test Data : ' + str(len(test_labels)) + ' Images')

Training Data : 61 Images
Validation Data : 10 Images
Test Data : 11 Images


## Ensemble the models

In [7]:
models = list()

In [8]:
vgg_model = load_model('/kaggle/input/landmark-identification-models/tensorflow2/final-models/2/model_vgg16.h5')
models.append(vgg_model)

In [9]:
resnet_model = load_model('/kaggle/input/landmark-identification-models/tensorflow2/final-models/2/model_Resnet.h5')
models.append(resnet_model)

## Results

In [10]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator()

In [11]:
test_generator = datagen.flow_from_directory(
            test_data_dir,
            target_size=(img_width, img_height),
            batch_size=len(test_labels),
            class_mode=None,
            shuffle=False)

Found 11 images belonging to 3 classes.


In [12]:
# make predictions
test_predictions = [model.predict(test_generator, 1, verbose=1) for model in models]
# calculate average
test_predictions = np.mean(test_predictions, axis=0)

1/1 [==============================] - 2s 2s/step


In [13]:
test_predictions = np.asarray(list(map(str,np.argmax(test_predictions,axis=1)))).reshape(-1,1)
test_predictions

array([['6'],
       ['6'],
       ['7'],
       ['7'],
       ['7'],
       ['7'],
       ['2'],
       ['3'],
       ['9'],
       ['7'],
       ['9']], dtype='<U1')

In [14]:
test_labels = np.asarray(test_labels).reshape(-1,1)
test_labels

array([['6'],
       ['6'],
       ['7'],
       ['7'],
       ['7'],
       ['7'],
       ['7'],
       ['9'],
       ['9'],
       ['9'],
       ['9']], dtype='<U1')

In [15]:
# Create an instance of the Accuracy metric
accuracy_metric = tf.keras.metrics.Accuracy()

# Update the state of the accuracy metric with true labels and predictions
test_labels = [float(pred) for pred in test_labels]
test_predictions = [float(pred) for pred in test_predictions]
accuracy_metric.update_state(test_labels, test_predictions)

# Get the test accuracy result
accuracy_result = accuracy_metric.result()
print("Testing Accuracy:", accuracy_result.numpy())

Testing Accuracy: 0.72727275
